<a href="https://colab.research.google.com/github/carloslme/data-engineering/blob/main/bucket_migration_s3_to_endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook has the code to perform a bucket migration from AWS S3 to an endpoint, for this case, Wasabisys (also called Wasabi).

# Contents
* Prerequisites
* Step 1: Import libraries
* Step 2: Setting credentials
* Step 3: Declare functions 
* Step 4: Get the S3 objects in the buckets
* Step 5: Start migration

# Prerequisites
Before you begin, be sure that you've installed `boto3`

In [1]:
!pip install boto3

     |████████████████████████████████| 131 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 7.8 MB 38.1 MB/s 
     |████████████████████████████████| 138 kB 45.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Step 1: Import libraries
Just importing the library installed and some preinstalled ones in notebook session

In [2]:
import boto3
import multiprocessing
from multiprocessing import Pool
import time
import io
from tqdm import tqdm

# Step 2: Setting credentials


In [ ]:
import boto3

# Setting up the credentials for S3
ACCESS_ID_RAW = ''
ACCESS_KEY_RAW = ''
AWS_S3_CREDS = {
         'aws_access_key_id':ACCESS_ID_RAW,
         'aws_secret_access_key':ACCESS_KEY_RAW
         }

# Setup up the credentiald for Wasabi
ACCESS_ID_W = ''
ACCESS_KEY_W = ''
ENPOINT_URL = 'https://s3.YOUR-REGION-HERE.wasabisys.com'
AWS_WASABI_CREDS = {
    'endpoint_url':ENPOINT_URL,
    'aws_access_key_id':ACCESS_ID_W,
    'aws_secret_access_key':ACCESS_KEY_W
}

# Step 3: Declare functions


In [ ]:
def get_all_s3_objects(s3, **base_kwargs):
    continuation_token = None
    while True:
        list_kwargs = dict(MaxKeys=1000, **base_kwargs)
        if continuation_token:
            list_kwargs['ContinuationToken'] = continuation_token
        response = s3.list_objects_v2(**list_kwargs)
        yield from response.get('Contents', [])
        if not response.get('IsTruncated'):  # At the end of the list?
            break
        continuation_token = response.get('NextContinuationToken')

def process_obj(obj_name):
  try:
    # Generating new client to avoid concurrency issues
    s3client = boto3.client('s3', **AWS_S3_CREDS)
    # Download object
    obj = s3client.get_object(Bucket='YOUR-BUCKET-NAME-HERE', Key=obj_name)
    # Upload object
    s3_wasabi.put_object(Bucket=bucket_name, Key='YOUR-BUCKET-NAME-HERE/' + obj_name, Body=io.BytesIO(obj['Body'].read()))
    return obj_name
  except Exception as e:
    return 'ERROR: YOUR-BUCKET-NAME-HERE/'+obj_name + ' | ' + e

def get_objects(bucket, prefix):
  response = get_all_s3_objects(boto3.client('s3', **AWS_S3_CREDS), Bucket=bucket, Prefix=prefix)
  return [o['Key'] for o in list(response)] # return object list 

# Step 4: Get the S3 objects in the buckets

In [ ]:
result_list = []
prefix = ''
bucket = ''
list_objs = get_objects(bucket, prefix)